# Introduction

#This kernel is for time series analysis to be used for ABB PoC.
Please add other demo model code below if anything is applicable with explaination

The order of explain is as follows.

-Overview of the data  

-EDA

-Data Preprocessing

-Model Building(Base line model with ARIMAX)

-Model evaluation

## Import Modules

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font='IPAGothic')
import numpy as np
import statsmodels.api as sm

In [5]:
folder_path = 'put import folder name here'
#sample = pd.read_csv(folder_path)

# EDA

Left this space for data analysis for insight check

## Data Preprocessing

Left this space for data preprocessing

# Model Building

In [6]:
#We can use seasonal_decompose method to separate into four graphs(Observed,Trend,Seasonal,Residual) as decomposition step

In [ ]:
res = sm.tsa.seasonal_decompose(buf.sales.dropna(),freq=365)
fig = res.plot()
fig.set_figheight(8)
fig.set_figwidth(15)
plt.show()

In [ ]:
#train_test_split
train_start_date = ''
train_end_date = ''
test_start_date = ''
test_end_Date = ''
tr_start,tr_end = train_start_date,train_end_date
te_start,te_end = test_start_date,test_end_Date
tra = sample['target'][tr_start:tr_end].dropna()
tes = sample['target'][te_start:te_end].dropna()

In [7]:
#We have to choice a model, After we comfirm that a data has a stationarity or not.

In [8]:
#We can use ADF-test to check stationary of the data.

In [ ]:
#ADF-test(Original-time-series)
df = final_df
res = sm.tsa.adfuller(final_df,regression='ct')
print('p-value:{}'.format(res[1]))

In [ ]:
#ADF-test(differenced-time-series)
df_diff = final_df.diff()
res = sm.tsa.adfuller(df_diff,regression='c')
print('p-value:{}'.format(res[1]))

In [ ]:
def test_stationarity(sample):
    
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=12)
    rolstd = pd.rolling_std(timeseries, window=12)

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print 'Results of Dickey-Fuller Test:'
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print dfoutput

In [ ]:
test_stationarity(sample)

In [9]:
#It's important to choose carefully a period of the data which will be used in predicting. Because, The results depend on the period.Check ADF_TEST for different period based on EDA

In [ ]:
#Autocorrelogram & Partail Autocorrelogram is useful that to estimate each models parametaers.

In [ ]:
#we use tra.diff()(differenced data), because this time series is unit root process.
fig,ax = plt.subplots(2,1,figsize=(20,10))
fig = sm.graphics.tsa.plot_acf(tra.diff().dropna(), lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(tra.diff().dropna(), lags=50, ax=ax[1])
plt.show()

In [10]:
#From results above,define like ARIMA(p=?,d=?,q=?) model.

#if we use arma_order_select_ic method, it is very easy to search best parameters(p,q) of ARMA model.

In [ ]:
resDiff = sm.tsa.arma_order_select_ic(tra, max_ar=7, max_ma=7, ic='aic', trend='c')
print('ARMA(p,q) =',resDiff['aic_min_order'],'is the best.')

In [11]:
#Based on above test, find the optimal p,d,q

In [12]:
#Check for seasonality in the time series, if with a seasonal order(0,0,0,0),then we can use SARIMAX or else go for ARIMAX

In [ ]:
#if seasonality is present then provide inputs in seasonal_order parameter
arimax = sm.tsa.statespace.ARIMAX(tra,order=(7,1,7),seasonal_order=(0,0,0,0),exog = exog_train,freq='D',
                                  enforce_stationarity=False, enforce_invertibility=False,).fit()
arimax.summary()
#We can use SARIMAX model as ARIMAX when seasonal_order is (0,0,0,0) .

## Model Evaluation

In [13]:
#after modelling check for residual 

In [ ]:
res = arima.resid
fig,ax = plt.subplots(2,1,figsize=(15,8))
fig = sm.graphics.tsa.plot_acf(res, lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(res, lags=50, ax=ax[1])
plt.show()

In [14]:
#check for MSE

In [ ]:
from sklearn.metrics import mean_squared_error
pred = arima.predict(tr_end,te_end)[1:]
print('ARIMA model MSE:{}'.format(mean_squared_error(tes,pred)))

In [ ]:
#test vs pred
pd.DataFrame({'test':tes,'pred':pred}).plot();plt.show()

In [15]:
#Make new feature based on above base line model results if needed

In [16]:
#rerun the model with hyperparameter tuning

In [ ]:
arimax.plot_diagnostics(figsize=(15, 12))